In [ ]:
import urllib.request
import pathlib
import shutil
import collections

import numpy as np
import pydicom
import matplotlib.pyplot as plt
import skimage.measure

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rai.model import load
from rai.data.images import paths_to_image_stack_hfs
from rai.mask.convert import contour_sequence_to_masks, mask_to_contours
from rai.metrics.dice import from_contours_by_slice

from raicontours import cfg, TG263

In [ ]:
model = load.load_model()

In [ ]:
data_path = pathlib.Path('data')

In [ ]:
# # TODO: This can be downloaded in parallel.

# data_path.mkdir(exist_ok=True)

# data_root = 'https://github.com/RadiotherapyAI/data-tcia-deepmind/raw/61fd2525f9880c8b201758f43c773e515572be92/0522c0659'

# filenames = [f"CT-{item:03d}.dcm" for item in range(165)] + ["RS.dcm"]

# for filename in filenames:
#     urllib.request.urlretrieve(f"{data_root}/{filename}", data_path / filename)

In [ ]:
image_paths = [data_path / f"CT-{item:03d}.dcm" for item in range(165)]

In [ ]:
# image_stack = paths_to_image_stack(image_paths)

In [ ]:
# image_stack.shape

In [ ]:
# np.max(image_stack)

In [ ]:
x_grid, y_grid, image_stack, image_uids = paths_to_image_stack_hfs(image_paths)


# initial_reduce_block_size = cfg["reduce_block_sizes"][0]

# reduced_image_stack = skimage.measure.block_reduce(
#     image_stack, block_size=initial_reduce_block_size, func=np.mean
# )
# reduced_x_grid = skimage.measure.block_reduce(
#     x_grid, block_size=initial_reduce_block_size[2], func=np.mean
# )
# reduced_y_grid = skimage.measure.block_reduce(
#     y_grid, block_size=initial_reduce_block_size[1], func=np.mean
# )

In [ ]:
structure_ds = pydicom.read_file(data_path / "RS.dcm")

In [ ]:
name_to_number_map = {
    item.ROIName: item.ROINumber for item in structure_ds.StructureSetROISequence
}

name_to_number_map

In [ ]:
TG263_to_deepmind_map = {
    TG263.Eye_L: 'Orbit-Lt',
    TG263.Eye_R: 'Orbit-Rt',
    TG263.OpticNrv_L: 'Optic-Nerve-Lt',
    TG263.OpticNrv_R: 'Optic-Nerve-Rt',
}

In [ ]:
number_to_contour_sequence_map = {
    item.ReferencedROINumber: item.ContourSequence for item in structure_ds.ROIContourSequence
}

In [ ]:
structure_name_to_contour_sequence_map = {
    structure_name: number_to_contour_sequence_map[name_to_number_map[TG263_to_deepmind_map[structure_name]]] for structure_name in cfg["structures"]
}

In [ ]:
structure_name_to_contour_sequence_map

In [ ]:
masks = []

for structure_name in cfg["structures"]:
    mask = contour_sequence_to_masks(x_grid, y_grid, image_uids, structure_name_to_contour_sequence_map[structure_name], expansion=4)
    masks.append(mask[..., None])
    
masks = np.concatenate(masks, axis=-1)

In [ ]:
masks.shape

In [ ]:
# shrunk_masks =  skimage.measure.block_reduce(
#     masks, block_size=(2, 4, 4, 1), func=np.max
# )

In [ ]:
# shrunk_masks.shape

In [ ]:
# image_uids

In [ ]:
gt_masks = masks[13:77, 145:209, 224:288, :]
np.max(gt_masks[..., 0])

In [ ]:
(200 + 155) / 2

In [ ]:
177 - 32

In [ ]:
177 + 32

In [ ]:
(227 + 286) / 2

In [ ]:
np.where(masks[..., 0] == 255)

In [ ]:
np.where(masks[..., 1] == 255)

In [ ]:
np.where(masks[..., 2] == 255)

In [ ]:
np.where(masks[..., 3] == 255)

In [ ]:
256 - 32

In [ ]:
256 + 32

In [ ]:
# reduced_image_stack.shape

In [ ]:
model_input = image_stack[None, 13:77, 145:209, 224:288]
model_input.shape

In [ ]:
model_output = model.predict(model_input)

In [ ]:
cfg["structures"]

In [ ]:
contours_by_structure_pd = {}
contours_by_structure_gt = {}

for structure_index, structure_name in enumerate(cfg["structures"]):
    this_structure_pd = model_output[0, ..., structure_index]
    this_structure_gt = gt_masks[..., structure_index]
    
    contours_by_slice_pd = []
    contours_by_slice_gt = []
    for z_index in range(64):
        this_slice_pd = this_structure_pd[z_index, ...]
        this_slice_gt = this_structure_gt[z_index, ...]
        
        contours_pd = mask_to_contours(x_grid, y_grid, this_slice_pd)
        contours_gt = mask_to_contours(x_grid, y_grid, this_slice_gt)
        
        contours_by_slice_pd.append(contours_pd)
        contours_by_slice_gt.append(contours_gt)
        
    contours_by_structure_pd[structure_name] = contours_by_slice_pd
    contours_by_structure_gt[structure_name] = contours_by_slice_gt

In [ ]:
# contours_by_structure_pd

In [ ]:
dice = {}

for structure_name in cfg["structures"]:
    a = contours_by_structure_pd[structure_name]
    b = contours_by_structure_gt[structure_name]
    
    dice[structure_name] = from_contours_by_slice(a, b)


dice

In [ ]:
def _plot_model_result(
    model_input, model_output
):
    contours_by_slice = _get_contours_by_slice(
        images=model_input,
        predictions=model_output,
    )

    x = list(range(model_input.shape[1]))
    vmin = 0.2
    vmax = 0.4

    ylim = [-np.inf, np.inf]
    xlim = [np.inf, -np.inf]

    axs = []

    for z_index, contours_by_name in contours_by_slice.items():
        fig, ax = plt.subplots()
        axs.append(ax)

        ax.pcolormesh(
            x,
            x,
            model_input[z_index, :, :],
            vmin=vmin,
            vmax=vmax,
            shading="nearest",
            cmap="gray",
        )

        for structure_name, contours in contours_by_name.items():
            for contour in contours:
                ax.plot(
                    contour[:, 1],
                    contour[:, 0],
                    label=structure_name.value,
                )

                xlim[1] = np.max([np.max(contour[:, 1]), xlim[1]])
                xlim[0] = np.min([np.min(contour[:, 1]), xlim[0]])
                ylim[0] = np.max([np.max(contour[:, 0]), ylim[0]])
                ylim[1] = np.min([np.min(contour[:, 0]), ylim[1]])

        ax.set_aspect("equal", "box")

        plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

    x_range = xlim[1] - xlim[0]
    y_range = ylim[0] - ylim[1]

    margin = 0.2

    xlim[0] -= x_range * margin
    xlim[1] += x_range * margin

    ylim[1] -= y_range * margin
    ylim[0] += y_range * margin

    for ax in axs:
        ax.set_ylim(ylim)
        ax.set_xlim(xlim)

    plt.show()

In [ ]:
def _get_contours_by_slice(
    images, predictions
):
    num_slices = images.shape[0]
    contours_by_slice = collections.defaultdict(dict)

    for i in range(num_slices):
        for structure_name in cfg["structures"]:
            k = cfg["structures"].index(structure_name)

            prediction_slice = predictions[i, :, :, k]

            if np.max(prediction_slice) < 127.5:
                continue

            contours = skimage.measure.find_contours(prediction_slice, level=127.5)
            contours_by_slice[i][structure_name] = contours

    return contours_by_slice

In [ ]:
_plot_model_result(model_input[0, ...], model_output[0, ...])